In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import plotly.express as px
import pandas as pd
import base64

# Function to convert age string to weeks
def convert_age_to_weeks(age_str):
    """Convert age string to weeks"""
    if pd.isna(age_str):
        return None
    try:
        num, unit = age_str.split()
        num = float(num)
        unit = unit.lower()
        if 'year' in unit:
            return num * 52
        elif 'month' in unit:
            return num * 4.34524  # Average weeks in month
        elif 'week' in unit:
            return num
        elif 'day' in unit:
            return num / 7
        return None
    except:
        return None

# Initialize app
app = JupyterDash(__name__)

# Load data from CSV file with error handling
try:
    print("Loading data from CSV file...")
    df = pd.read_csv('aac_shelter_outcomes.csv')
    
    # Ensure required columns exist (add if missing)
    required_columns = [
        'name', 'animal_type', 'breed', 'age_upon_outcome',
        'sex_upon_outcome', 'location_lat', 'location_long'
    ]
    
    for col in required_columns:
        if col not in df.columns:
            df[col] = None  # Add missing columns with null values
    
    print(f"Successfully loaded {len(df)} records from CSV")
    
    # Convert age to weeks for filtering
    if 'age_upon_outcome' in df.columns:
        df['age_upon_outcome_in_weeks'] = df['age_upon_outcome'].apply(
            lambda x: convert_age_to_weeks(x) if pd.notna(x) else None
        )
    
except Exception as e:
    print(f"CSV loading error: {str(e)}")
    print("Using empty DataFrame as fallback")
    df = pd.DataFrame(columns=required_columns)
    df['age_upon_outcome_in_weeks'] = None

# Define rescue filters
RESCUE_FILTERS = {
    'water': {
        'animal_type': 'Dog',
        'breed': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'],
        'sex_upon_outcome': 'Intact Female',
        'age_upon_outcome_in_weeks': lambda x: (x >= 26) & (x <= 156)
    },
    'mountain': {
        'animal_type': 'Dog',
        'breed': ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'],
        'sex_upon_outcome': 'Intact Male',
        'age_upon_outcome_in_weeks': lambda x: (x >= 26) & (x <= 156)
    },
    'disaster': {
        'animal_type': 'Dog',
        'breed': ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'],
        'sex_upon_outcome': 'Intact Male',
        'age_upon_outcome_in_weeks': lambda x: (x >= 20) & (x <= 300)
    }
}

# Encode logo image
try:
    logo_path = 'Grazioso Salvare Logo.png'
    logo_encoded = base64.b64encode(open(logo_path, 'rb').read()).decode()
except Exception as e:
    print(f"Logo loading error: {str(e)}")
    logo_encoded = ""

# App layout
app.layout = html.Div([
    # Header with logo and identifier
    html.Div([
        html.Img(src=f'data:image/png;base64,{logo_encoded}', 
                style={'height': '100px', 'float': 'left'}),
        html.H1("Animal Rescue Dashboard", style={'text-align': 'center'}),
        html.Div("Created by Jamar Sampson",
                style={'text-align': 'right', 'font-style': 'italic'})
    ], style={'margin-bottom': '20px'}),
    
    # Filter controls
    dcc.RadioItems(
        id='rescue-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain Rescue', 'value': 'mountain'},
            {'label': 'Disaster Rescue', 'value': 'disaster'},
            {'label': 'All Animals', 'value': 'all'}
        ],
        value='all',
        labelStyle={'display': 'inline-block', 'margin-right': '10px'}
    ),
    
    # Data table with enhanced features
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": col, "id": col} for col in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        page_action='native',
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        style_table={'overflowX': 'auto'},
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        },
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px', 'width': '150px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        }
    ),
    
    # Charts and map
    html.Div([
        dcc.Graph(id='breed-chart'),
        dl.Map(
            id='map',
            style={'height': '400px', 'margin-top': '20px'},
            center=[30.2672, -97.7431],
            zoom=10,
            children=[dl.TileLayer()]
        )
    ])
])

# Callbacks
@app.callback(
    Output('data-table', 'data'),
    Input('rescue-filter', 'value')
)
def update_table(filter_value):
    if filter_value == 'all':
        return df.to_dict('records')
    
    # Apply filters for CSV data
    filtered_df = df.copy()
    filters = RESCUE_FILTERS[filter_value]
    
    # Apply each filter condition
    for col, condition in filters.items():
        if col == 'breed':
            filtered_df = filtered_df[filtered_df[col].isin(condition)]
        elif col == 'age_upon_outcome_in_weeks':
            filtered_df = filtered_df[condition(filtered_df[col])]
        else:
            filtered_df = filtered_df[filtered_df[col] == condition]
    
    return filtered_df.to_dict('records')

@app.callback(
    Output('breed-chart', 'figure'),
    Input('data-table', 'derived_virtual_data')
)
def update_chart(data):
    if not data or len(data) == 0:
        return px.pie(title="No data available")
    breed_counts = pd.DataFrame(data)['breed'].value_counts().head(10).reset_index()
    return px.pie(breed_counts, names='index', values='breed', title='Top 10 Breeds')

@app.callback(
    Output('map', 'children'),
    Input('data-table', 'derived_virtual_data')
)
def update_map(data):
    if not data:
        return [dl.TileLayer()]
    
    df_map = pd.DataFrame(data)
    markers = [
        dl.Marker(
            position=[row['location_lat'], row['location_long']],
            children=[dl.Tooltip(row['name'])]
        )
        for _, row in df_map.iterrows() 
        if pd.notna(row.get('location_lat')) and pd.notna(row.get('location_long'))
    ]
    return [dl.TileLayer()] + markers

app.run_server(mode='inline')

Loading data from CSV file...
Successfully loaded 10000 records from CSV
